# Upload data

In [1]:
import pandas as pd
import GEOparse
import numpy as np
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

In [2]:
gse = GEOparse.get_GEO(geo="GSE26728", destdir="./")

05-Apr-2020 23:24:22 DEBUG utils - Directory ./ already exists. Skipping.
05-Apr-2020 23:24:22 INFO GEOparse - File already exist: using local version.
05-Apr-2020 23:24:22 INFO GEOparse - Parsing ./GSE26728_family.soft.gz: 
05-Apr-2020 23:24:22 DEBUG GEOparse - DATABASE: GeoMiame
05-Apr-2020 23:24:22 DEBUG GEOparse - SERIES: GSE26728
05-Apr-2020 23:24:22 DEBUG GEOparse - PLATFORM: GPL7042
/home/for_x_in_rage/anaconda3/lib/python3.7/site-packages/GEOparse/GEOparse.py:104: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  return parse_GSE(filepath)
05-Apr-2020 23:24:23 DEBUG GEOparse - SAMPLE: GSM658075
05-Apr-2020 23:24:23 DEBUG GEOparse - SAMPLE: GSM658076
05-Apr-2020 23:24:23 DEBUG GEOparse - SAMPLE: GSM658077
05-Apr-2020 23:24:23 DEBUG GEOparse - SAMPLE: GSM658078
05-Apr-2020 23:24:23 DEBUG GEOparse - SAMPLE: GSM658079
05-Apr-2020 23:24:23 DEBUG GEOparse - SAMPLE: GSM658080
05-Apr-2020 23:24:23 DEBUG GEOparse - SAMPLE: GSM658081
05-

In [3]:
control_name = ['GSM658075','GSM658076','GSM658077','GSM658078','GSM658079','GSM658080']
low_dose_name = ['GSM658081','GSM658082','GSM658082','GSM658084','GSM658085','GSM658086']
high_dose_name = ['GSM658087','GSM658088','GSM658089','GSM658090','GSM658091','GSM658092']

# Количество генов в исследовании

**22514** гена,  дупликатов нет

In [4]:
num_of_samples = []

control_list, low_dose_list, high_dose_list = [], [], []

# print number of unique genes in experiments
# and separate data by groups: control, low dose, high dose
for gsm_name, gsm in gse.gsms.items():
    
    num_genes = gsm.table.shape[0]
    num_unique = len(gsm.table.ID_REF.unique())
    
    if gsm_name in control_name:
        control_list.append(gsm.table.set_index('ID_REF'))
    elif gsm_name in low_dose_name:
        low_dose_list.append(gsm.table.set_index('ID_REF'))
    elif gsm_name in high_dose_name:
        high_dose_list.append(gsm.table.set_index('ID_REF'))
    
    if num_genes in num_of_samples:
        continue
    
    num_of_samples.append(num_genes)

    print('Number of genes: ', num_genes)
    print('Number of unique samples: ', num_unique)

Number of genes:  22514
Number of unique samples:  22514


In [5]:
# Create dataframes for each group of samples
# containing all the data for each gene
control = pd.concat(control_list, axis = 0)
low_dose = pd.concat(low_dose_list, axis = 0)
high_dose = pd.concat(high_dose_list, axis = 0)

# Cредние значения экспресии каждого из генов в контроле, low_dose и high_dose. 

In [6]:
# Dataframes with mean values of expression for each gene in each group
control_mean = control.groupby(by='ID_REF').mean()
low_dose_mean = low_dose.groupby(by='ID_REF').mean()
high_dose_mean = high_dose.groupby(by='ID_REF').mean()

# Построение классификаторов control vs low_dose, control vs high_dose

In [7]:
def create_dataset(dose, control):
    '''Function that transform dataframes with dose and genes into X,y dataset
    '''
    dose = dose.transpose()
    control = control.transpose()
    dataset = pd.concat([dose,control], axis = 0)
    dataset['Y'] = np.array([0]*len(dose)+[1]*len(control))
    dataset = dataset.sample(frac=1)
    y=dataset['Y']
    X=dataset.drop(columns=['Y'],axis=1)
    return X,y

In [8]:
# prepare dataframe
control_dataset = pd.concat(control_list, axis = 1, join='inner')
low_dose_dataset = pd.concat(low_dose_list, axis = 1, join='inner')
high_dose_dataset = pd.concat(high_dose_list, axis = 1, join='inner')

In [18]:
X_low,y_low = create_dataset(low_dose_dataset, control_dataset)

In [22]:
classification_tree_low = RandomForestClassifier(n_estimators=3000, max_depth = 2)

In [ ]:
classification_tree_low = classification_tree.fit(X_low,y_low)

In [ ]:
np.sort(classification_tree_low.feature_importances_)[-40:]